# Модуль 9.1 — Named Entity Recognition (NER)

**Цель:** извлекать сущности (имена, организации, даты, локации) из текста с помощью spaCy, natasha и Hugging Face.

**Что сделаем:**
- установим библиотеки и загрузим модели
- запустим spaCy (EN/RU)
- попробуем natasha (специально для русского)
- используем NER-модели из Hugging Face

---
### 🎯 Якорь
Выделить имена, организации, даты в тексте — как маркером пометить «здесь персона, здесь место».

**Термины:** NER | spaCy (ru_core_news_sm) | natasha (русский) | PERSON, ORG, LOC, DATE

## Установка зависимостей

В Colab: после `pip install` может понадобиться **Runtime → Restart session**.

In [ ]:
%pip install -q spacy transformers torch
%pip install -q natasha  # русский NER и морфология
# spaCy модели (выберите одну или обе):
# !python -m spacy download en_core_web_sm   # английский
!python -m spacy download ru_core_news_sm   # русский

## 1. spaCy NER

spaCy — быстрый и удобный пайплайн для NER. Поддерживает русский (`ru_core_news_sm`) и английский (`en_core_web_sm`).

In [ ]:
import spacy

# Русская модель
nlp_ru = spacy.load("ru_core_news_sm")

text = "Иван Петров из компании Яндекс встретился в Москве с главой Роснефти Игорем Сечиным 15 марта 2024 года."
doc = nlp_ru(text)

print("spaCy (ru_core_news_sm):")
for ent in doc.ents:
    print(f"  {ent.text:<25} {ent.label_}")

## 2. natasha (русский)

natasha заточен под русский медиа-текст. Использует морфологию и именованные сущности.

In [ ]:
from natasha import (
    Segmenter, NewsEmbedding, NewsMorphTagger, NewsNERTagger, Doc
)

segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
ner_tagger = NewsNERTagger(emb)

doc = Doc(text)
doc.segment(segmenter).tag_morph(morph_tagger).tag_ner(ner_tagger)

print("natasha:")
for span in doc.spans:
    print(f"  {span.text:<25} {span.type}")

## 3. Hugging Face NER

Модели из Hugging Face дают гибкость: можно взять специализированную модель под домен (юр. документы, новости и т.д.).

In [ ]:
from transformers import pipeline

# Русская NER-модель (Gherman/bert-base-NER-Russian)
ner = pipeline(
    "ner",
    model="Gherman/bert-base-NER-Russian",
    aggregation_strategy="simple"
)

results = ner(text)
print("Hugging Face (bert-base-NER-Russian):")
for r in results:
    lbl = r.get("entity_group") or r.get("entity", "").split("-")[-1]
    print(f"  {r['word']:<25} {lbl}")

## Микро-упражнение

Запусти один и тот же текст через spaCy и natasha, сравни результаты. Какой инструмент лучше распознал имена и организации?